In [1]:
import azureml.core
from azureml.core import Workspace

# Load the workspace from the saved config file
ws = Workspace.from_config()
print('Ready to use Azure ML {} to work with {}'.format(azureml.core.VERSION, ws.name))

Failure while loading azureml_run_type_providers. Failed to load entrypoint automl = azureml.train.automl.run:AutoMLRun._from_run_dto with exception (portalocker 2.0.0 (/anaconda/envs/azureml_py36/lib/python3.6/site-packages), Requirement.parse('portalocker~=1.0'), {'msal-extensions'}).


Ready to use Azure ML 1.11.0 to work with mlops-aml


In [2]:
from azureml.core import Model

for model in Model.list(ws):
    print(model.name, 'version:', model.version)
    for tag_name in model.tags:
        tag = model.tags[tag_name]
        print ('\t',tag_name, ':', tag)
    for prop_name in model.properties:
        prop = model.properties[prop_name]
        print ('\t',prop_name, ':', prop)
    print('\n')

porto_seguro_safe_driver_model version: 3
	 learning_rate : 0.02
	 boosting_type : gbdt
	 objective : binary
	 metric : auc
	 sub_feature : 0.7
	 num_leaves : 60
	 min_data : 100
	 min_hessian : 1
	 verbose : 0
	 auc : 0.6377511613946426


driver_model.pkl version: 2
	 auc : 0.6377511613946426


porto_seguro_safe_driver_model version: 2
	 learning_rate : 0.02
	 boosting_type : gbdt
	 metric : auc
	 objective : binary
	 sub_feature : 0.7
	 min_data : 100
	 num_leaves : 60
	 min_hessian : 1
	 verbose : 0
	 auc : 0.6377511613946426


porto_seguro_safe_driver_model version: 1


driver_model.pkl version: 1
	 auc : 0.6377511613946426




In [3]:
model = ws.models['porto_seguro_safe_driver_model']
print(model.name, 'version', model.version)

porto_seguro_safe_driver_model version 3


In [4]:
import os

folder_name = 'driver-service'

# Create a folder for the web service files
experiment_folder = './' + folder_name
os.makedirs(folder_name, exist_ok=True)

print(folder_name, 'folder created.')

driver-service folder created.


In [5]:
from azureml.core.conda_dependencies import CondaDependencies 

# Add the dependencies for our model (AzureML defaults is already included)
myenv = CondaDependencies()
myenv.add_conda_package("scikit-learn")
myenv.add_conda_package("lightgbm")

# Save the environment config as a .yml file
env_file = folder_name + "/driver_env.yml"
with open(env_file,"w") as f:
    f.write(myenv.serialize_to_string())
print("Saved dependency info in", env_file)

# Print the .yml file
with open(env_file,"r") as f:
    print(f.read())

Saved dependency info in driver-service/driver_env.yml
# Conda environment specification. The dependencies defined in this file will
# be automatically provisioned for runs with userManagedDependencies=False.

# Details about the Conda environment file format:
# https://conda.io/docs/user-guide/tasks/manage-environments.html#create-env-file-manually

name: project_environment
dependencies:
  # The python interpreter version.
  # Currently Azure ML only supports 3.5.2 and later.
- python=3.6.2

- pip:
    # Required packages for AzureML execution, history, and data preparation.
  - azureml-defaults

- scikit-learn
- lightgbm
channels:
- anaconda
- conda-forge



In [6]:
from azureml.core.webservice import AciWebservice
from azureml.core.model import InferenceConfig

# Configure the scoring environment
inference_config = InferenceConfig(runtime= "python",
                                   source_directory = folder_name,
                                   entry_script="score.py",
                                   conda_file="driver_env.yml")

deployment_config = AciWebservice.deploy_configuration(cpu_cores = 1, memory_gb = 1)

service_name = "driver-service"

service = Model.deploy(ws, service_name, [model], inference_config, deployment_config)

service.wait_for_deployment(True)
print(service.state)

Running.................................
Succeeded
ACI service creation operation finished, operation "Succeeded"
Healthy


In [7]:
print(service.state)
print(service.get_logs())

Healthy
2020-08-28T20:56:34,888398179+00:00 - iot-server/run 
2020-08-28T20:56:34,889038976+00:00 - rsyslog/run 
2020-08-28T20:56:34,889506175+00:00 - gunicorn/run 
2020-08-28T20:56:34,891288068+00:00 - nginx/run 
/usr/sbin/nginx: /azureml-envs/azureml_06c16b148b04e8226cfe58e6ee379ca1/lib/libcrypto.so.1.0.0: no version information available (required by /usr/sbin/nginx)
/usr/sbin/nginx: /azureml-envs/azureml_06c16b148b04e8226cfe58e6ee379ca1/lib/libcrypto.so.1.0.0: no version information available (required by /usr/sbin/nginx)
/usr/sbin/nginx: /azureml-envs/azureml_06c16b148b04e8226cfe58e6ee379ca1/lib/libssl.so.1.0.0: no version information available (required by /usr/sbin/nginx)
/usr/sbin/nginx: /azureml-envs/azureml_06c16b148b04e8226cfe58e6ee379ca1/lib/libssl.so.1.0.0: no version information available (required by /usr/sbin/nginx)
/usr/sbin/nginx: /azureml-envs/azureml_06c16b148b04e8226cfe58e6ee379ca1/lib/libssl.so.1.0.0: no version information available (required by /usr/sbin/nginx)


In [8]:
for webservice_name in ws.webservices:
    print(webservice_name)

driver-service


In [9]:
import json

# This time our input is an array of two feature arrays
x_new = [[0,1,8,1,0,0,1,0,0,0,0,0,0,0,12,1,0,0,0.5,0.3,0.610327781,7,1,-1,0,-1,1,1,1,2,1,65,1,0.316227766,0.669556409,0.352136337,3.464101615,0.1,0.8,0.6,1,1,6,3,6,2,9,1,1,1,12,0,1,1,0,0,1],
    [4,2,5,1,0,0,0,0,1,0,0,0,0,0,5,1,0,0,0.9,0.5,0.771362431,4,1,-1,0,0,11,1,1,0,1,103,1,0.316227766,0.60632002,0.358329457,2.828427125,0.4,0.5,0.4,3,3,8,4,10,2,7,2,0,3,10,0,0,1,1,0,1]]

# Convert the array or arrays to a serializable list in a JSON document
input_json = json.dumps({"data": x_new})

# Call the web service, passing the input data
predictions = service.run(input_data = input_json)

# Get the predicted classes.
predicted_classes = predictions['result']
   
for i in range(len(x_new)):
    print ("Driver {}".format(x_new[i]), predicted_classes[i] )

Driver [0, 1, 8, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 12, 1, 0, 0, 0.5, 0.3, 0.610327781, 7, 1, -1, 0, -1, 1, 1, 1, 2, 1, 65, 1, 0.316227766, 0.669556409, 0.352136337, 3.464101615, 0.1, 0.8, 0.6, 1, 1, 6, 3, 6, 2, 9, 1, 1, 1, 12, 0, 1, 1, 0, 0, 1] 0.027311045841034335
Driver [4, 2, 5, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 5, 1, 0, 0, 0.9, 0.5, 0.771362431, 4, 1, -1, 0, 0, 11, 1, 1, 0, 1, 103, 1, 0.316227766, 0.60632002, 0.358329457, 2.828427125, 0.4, 0.5, 0.4, 3, 3, 8, 4, 10, 2, 7, 2, 0, 3, 10, 0, 0, 1, 1, 0, 1] 0.0261231327307869


In [10]:
service.delete()